In [1]:
!pip install autokeras -q

## 8.2.1 Data parallelism

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import autokeras as ak

(x_train, y_train), (x_test, y_test) = mnist.load_data()
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=1,
    distribution_strategy=tf.distribute.MirroredStrategy())
clf.fit(x_train, y_train, epochs=1)

Trial 1 Complete [00h 01m 54s]
val_loss: 0.061207693070173264

Best val_loss So Far: 0.061207693070173264
Total elapsed time: 00h 01m 54s
INFO:tensorflow:Oracle triggered exit
1875/1875 [==============================] - 133s 70ms/step - loss: 0.1572 - accuracy: 0.9515
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [3]:
import kerastuner as kt

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=1,
    directory='my_dir',
    distribution_strategy=tf.distribute.MirroredStrategy(),
    project_name='helloworld')

tuner.search(x_train, y_train,
             epochs=1,
             validation_data=(x_test, y_test))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Reloading Oracle from existing project my_dir/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/helloworld/tuner0.json
INFO:tensorflow:Oracle triggered exit


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


## 8.2.2 Model parallelism

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
import tensorflow as tf


tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


## 8.2.3 Parallel tuning

Note: The code below are only for the convenience of copying. It should be setup on your machines and run locally.

```shell
export KERASTUNER_TUNER_ID="chief"
export KERASTUNER_ORACLE_IP="127.0.0.1"
export KERASTUNER_ORACLE_PORT="8000"
python run_tuning.py
```

```shell
export KERASTUNER_TUNER_ID="tuner0"
export KERASTUNER_ORACLE_IP="127.0.0.1"
export KERASTUNER_ORACLE_PORT="8000"
python run_tuning.py
```

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import autokeras as ak
import kerastuner as kt

(x_train, y_train), (x_test, y_test) = mnist.load_data()

def build_model(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(
  units=hp.Int('units', min_value=32, max_value=512, step=32),
  activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
  return model

tuner = kt.RandomSearch(
  build_model,
  objective='val_loss',
  max_trials=1,
  directory='result_dir',
  project_name='helloworld')

tuner.search(x_train, y_train,
  epochs=1,
  validation_data=(x_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project result_dir/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from result_dir/helloworld/tuner0.json
INFO:tensorflow:Oracle triggered exit
